In [ ]:
import PyPDF2
import pandas as pd

pdfFileObj = open('ComputingScienceSummaries.pdf', 'rb')
pdfReader = PyPDF2.PdfReader(pdfFileObj)
  
n_pages = len(pdfReader.pages)
text = ''

for n in range(n_pages):
    pageObj = pdfReader.pages[n]
    text += pageObj.extract_text()
  
pdfFileObj.close()

In [ ]:
courses = {}
use_next_line = False
course_type = 'CSE'

for i, line in enumerate(text.splitlines()):
    if line.strip().startswith(course_type):
        course = line.replace('  ', ' ').strip()
    elif 'Prerequisite' in line:
        preq = line.split(': ')[1].strip().replace(' ', '')
        if 'None' in preq:
            preq = ''
        courses[course] = [preq]
    if use_next_line:
        
        #print('True', line)
        preq2 = line.split(': ')[0].strip().replace(' ', '')
        if len(preq2) != 0: # in case there is no prereq on this line
            preq = preq + ',' + preq2
        courses[course] = [preq]
        use_next_line = False
    if 'Prerequisites' in line:
        #print('use next line', line)
        #print(i, course, preq, '---', line)
        use_next_line = True
    #if '1120' in line:
    #if '1010' in line:
    #    print(i, use_next_line, line)
df = pd.DataFrame.from_dict(courses, orient='index', columns=['Prerequisites']).reset_index()
df.columns = ['Course', 'Prerequisites']
df

In [ ]:
df[df['Prerequisites'].str.contains('CSE')]

In [ ]:
df[df['Course'].str[3]=='2']

In [ ]:
df.to_csv(course_type+'.csv', index=False)

In [ ]:
df = pd.read_csv('CSE.csv')
df.fillna('', inplace=True)
df

In [ ]:
import graphviz
dot = graphviz.Digraph()

for row in df.itertuples():
    course_number = row.Course.split(':')[0]
    if '50' not in course_number:
        if course_number[3] == '3':
            dot.node(course_number, shape='diamond')
        else:
            dot.node(course_number, shape='box')
        if row.Prerequisites != '':
            for preq in row.Prerequisites.split(','):
                dot.edge(preq, course_number)

dot
# save the output to a file
dot.render(course_type, view=True)